In [1]:
import sys
from pathlib import Path
import import_ipynb
import numpy as np
import gmsh
from scipy.sparse.linalg import spsolve
from analytical_solution import potential_and_gradient

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    from pre_processing import domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results, error_evaluation
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 1: Poisson Problem at $L$ Domain   

Considere a geometria L apresentada na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/L_domain.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of L domain.</figcaption>
</figure>

Não há cargas no domínio e a solução exata para o problema é

$$
V(r, \theta) = r^{2/3} sin \left( \frac{2\theta}{3} + \frac{\pi}{3} \right)
\tag{1}
$$

onde $r$ e $\theta$ são as coordenadas polares, 

$$
\begin{cases}
r = \sqrt{x^2 + y^2}  \\
\theta = arctan(y/x) \tag{2}
\end{cases}
$$

A variação de $\theta$ para a geometria é $-\pi/2 \leq \theta \leq \pi$.

As condições de contorno são todas de Dirichlet e podem ser calculadas utilizando a expressão da solução exata sobre as fronteiras.

Usar malhas com densidades diferentes e verificar a convergência da sua solução à medida que refinna a malha. Calcule os erros na norma $L_2$ e na norma de energia. A melhor forma de verificar a convergência é traçar gráfios em escala _log-log_ da norma do erro em função de h. Se tudo estiver correto, em escala _log-log_ você obterá retas cujas inclinações fornecerão as taxas de convergência.

# `setTypeProblem()`

In [2]:
def setTypeProblem(element_type, lc):
    mesh_data = {}
    gmsh.initialize()
    
    BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'entire_boundary'}]
    MATERIAL = [{'type': 'material_constant_a', 'tag': 201, 'value': 1, 'name': 'entire_domain'}]
    domain.get_L_geometry(element_type, BOUNDARY, lc, auto_save=False)
    
    # Create mesh Structure Data from gmsh
    mesh_data['nodes_data'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['conn_data'] = read_mesh.get_conn_data(MATERIAL)
    gmsh.finalize()  

    # Set FEM Problem
    rho = np.zeros(len(mesh_data['conn_data']))
    potential_function = lambda x, y: potential_and_gradient(x, y)[0]
    boundary_conditions.assign_dirichlet_potential(mesh_data, potential_function)
    Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, rho, element_type)
    ur = spsolve(Agr.tocsr(), fgr.toarray())
    u = boundary_conditions.global_potential_vector(mesh_data, ur)

    return u, mesh_data

# `convergence()`

In [3]:
element_types = [('Triangle', 1), ('Triangle', 2), ('Triangle', 3),
                 ('Quadrangle', 1), ('Quadrangle', 2)]
analytical_solution = lambda x, y: potential_and_gradient(x, y)
mesh_sizes = [1 / (2**i) for i in range(2, 5)]

# Inicialização dos dicionários para armazenar erros e capacitâncias
errors_l2 = {et: [] for et in element_types}
errors_energy = {et: [] for et in element_types}
capacitances = {et: [] for et in element_types}

for lc in mesh_sizes:  # Loop para cada refinamento de malha
    for element_type in element_types:  # Loop para cada tipo de elemento
        # Resolve a solução numérica u_h para o tipo de problema
        uh, mesh_data = setTypeProblem(element_type, lc)  
        
        # Calcula o erro L2, o erro de energia e a energia total
        l2, energy, total_energy = error_evaluation.get_errors_and_energy(
            mesh_data, element_type, uh, analytical_solution)

        # Acumula os resultados nos dicionários correspondentes
        errors_l2[element_type].append(l2)
        errors_energy[element_type].append(energy)

# Graphics

In [4]:
graph_results.plot_convergence_with_slope(mesh_sizes, errors_l2, model='L', name='L2')
graph_results.plot_convergence_with_slope(mesh_sizes, errors_energy, model='L', name='Energy')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\3_L_domain\pos_processing\pictures\L_convergence_L2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\3_L_domain\pos_processing\pictures\L_convergence_Energy.svg


### Figure 1: Error Convergence for $L_2$ norm
<figure>
    <img src="pos_processing/pictures/L_convergence_L2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Convergence of the numerical solution for $L_2$ norm of the error.</figcaption>
</figure>

### Figure 2: Error Convergence for _Energy_ norm
<figure>
    <img src="pos_processing/pictures/L_convergence_Energy.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Convergence of the numerical solution for Energy norm of the error.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.